In [8]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters langchain-chroma langchain-core langchain-upstage
%pip install -U langchain langchainhub --quiet
%pip install -qU langchain langchain-pinecone langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # Chunk 한개가 가질 수 있는 토큰 수
    chunk_overlap=300, # Chunk 를 겹치게 자르는, 1 - 3, 다음은 2 - 4 이런식으로
)

# 문서를 가져와서,
loader = Docx2txtLoader('./tax_with_markdown.docx')

# Chunk 단위로 문서를 쪼갠다
document_list = loader.load_and_split(
    text_splitter=text_splitter
)

In [31]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatUpstage()
#llm = ChatOpenAI(model='gpt-4o')

embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

- Chroma : Vector in-memory Database

In [32]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
index_name = "tax-markdown-index"

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # Chunk 한개가 가질 수 있는 토큰 수
    chunk_overlap=100, # Chunk 를 겹치게 자르는, 1 - 3, 다음은 2 - 4 이런식으로
)

chunked_documents = text_splitter.split_documents(document_list)

database = PineconeVectorStore.from_documents(
    documents=[],
    embedding=embedding,
    index_name=index_name
)

batch_size = 100
for i in range(0, len(chunked_documents), batch_size):
    print(f'index: {i}, batch size : {batch_size}')
    batch = chunked_documents[i: i + batch_size]
    database.add_documents(batch)

index: 0, batch size : 100
index: 100, batch size : 100
index: 200, batch size : 100
index: 300, batch size : 100
index: 400, batch size : 100


In [46]:
query = '연봉 5천만원 초과인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query, k = 3)
print(retrived_docs)

[Document(id='b8c54eef-55da-49dd-b3ec-e02b8d2944ce', metadata={'source': './tax_with_markdown.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n[제목개정 2014. 1. 1.]\n\n\n\n제4절 세액의 계산 <개정 2009. 12. 31.>\n\n\n\n제1관 세율 <개정 2009. 12. 31.>\n\n\n\n제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 

In [47]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [48]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs={"prompt" : prompt}
)

ai_message = qa_chain.invoke({"query" : query})
print(ai_message)

{'query': '연봉 5천만원 초과인 직장인의 소득세는 얼마인가요?', 'result': '연봉 5천만원 초과인 직장인의 소득세는 종합소득 과세표준에 따라 달라집니다. 5천만원 초과 8,800만원 이하의 경우, 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)를 적용하여 계산한 금액이 소득세가 됩니다. 예를 들어, 연봉이 6천만원인 경우, (6천만원 - 5천만원) * 24% = 48만원을 624만원에 더한 672만원이 소득세가 됩니다.'}


In [49]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
다음 문맥을 바탕으로 질문에 답하세요.
<context>
{context}
</context>

질문: {input}

""")

combine_docs_chain = create_stuff_documents_chain(llm, prompt)

retriever = database.as_retriever()
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

response = rag_chain.invoke({"input": query})
print(response["answer"])

연봉 5천만원을 초과하는 직장인의 소득세는 해당 직장인의 종합소득과세표준에 따라 다르게 계산됩니다. 위의 표에서 5천만원 초과 8천8백만원 이하의 종합소득과세표준을 가진 직장인의 경우, 소득세는 다음과 같이 계산됩니다.

- 624만원 + (5천만원을 초과하는 금액의 24퍼센트)

예를 들어, 연봉이 6천만원인 경우를 계산해보면:

- 6천만원 - 5천만원 = 1천만원 (5천만원을 초과하는 금액)
- 1천만원 × 24% = 240만원
- 624만원 + 240만원 = 864만원

따라서, 연봉 6천만원을 초과하는 직장인의 경우, 소득세는 864만원이 됩니다. 그러나 이는 일반적인 계산 방법이며, 실제 소득세는 다양한 공제 및 세액 조정 사항에 따라 달라질 수 있습니다. 정확한 소득세 계산을 위해서는 개인의 구체적인 상황을 고려해야 합니다.
